# __[전처리 + 키워드 추출 + 집계]__

\* Keyphrase, candidats 등 문맥을 고려해 '키워드'로 통칭

### ※기타 유용한 환경 구성

In [1]:
# 셀 별로 러닝타임 측정
# !pip install ipython-autotime
%load_ext autotime

time: 0 ns (started: 2023-08-28 21:56:40 +09:00)


In [2]:
# GPU 사용 환경 구성
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

time: 0 ns (started: 2023-08-28 21:56:42 +09:00)


---

## ■ 불용어 전처리

In [ ]:
cd "../Data"

In [5]:
# 전처리 코드
import re
import pandas as pd

# 텍스트 정규화 수행: 같은 의미지만 다른 표현으로 쓰이는 경우가 있기에, 연산량을 줄이고자 정규화 과정이 필요

contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                "you're": "you are", "you've": "you have"}

print("정규화 사전의 수: ", len(contractions))

def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\d+ml|\d+ ml', '', sentence)  # "ml"을 포함하는 문자 제거
    sentence = re.sub(r'\d+oz|\d+ oz', '', sentence)  # "oz"을 포함하는 문자 제거
    sentence = re.sub(r'\d+cl|\d+ cl', '', sentence)  # "cl"을 포함하는 문자 제거
    sentence = re.sub(r'---rated via beer buddy for iphone|---rated via beerbuddy for iphone', '', sentence)  # 특정 문구 제거
    sentence = re.sub(r'on tap[ .,?!]|tap[ .,?!]', '', sentence)  # "on tap, tap"을 포함하는 문자 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (.) 제거
    sentence = re.sub(r'[.,?!]+[/.,?!]', '', sentence) # 여러개 자음, 모음, 구두점 제거
    sentence = re.sub(r"[^a-z0-9.,!?'"":]", " ", sentence) # 지정한 문자 제외 공백으로 전환
    sentence = re.sub(r"[\d\/\d\/\d]", "", sentence) # 날짜 삭제
    sentence = re.sub(r"bottle[ .,?!:]|bottled[[ .,?!:]]", "", sentence) # bottle 관련 삭제
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")]) # 약어 정규화
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿈
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    return sentence

# 수정 및 추가 사항
# on tap[특수기호] 형태 다수 포착하여 처리
# bottle[특수기호] 형태 다수 포착하여 처리 (+bottled 필터 추가)
# 카테고리별로 리뷰한 데이터에 ':'가 삭제되어 카테고리+리뷰가 나옴 
# (원문 Appearance: pitch black, Flavour: biscuit, coffee, ...)
# (추출 appearance pitch black, flavour biscuit)

정규화 사전의 수:  120
time: 359 ms (started: 2023-08-28 21:57:18 +09:00)


In [6]:
# 데이터 불러오기
# 전처리 전, 후 비교를 위해 복사본 생성
df = pd.read_csv('Binary Classification_v3.csv')
              
# 맥주 하나 지정
# selected_reviews = df[df['Beer_name'] == '8 Wired iStout']

# 맥주 여러개 지정 필터링
beer_names = ['Asahi Super Dry', '8 Wired iStout', 'Red Rock']
selected_reviews = df[df['Beer_name'].isin(beer_names)]

selected_reviews.reset_index(drop = True, inplace = True)
pre_apply_selected_reviews = selected_reviews.copy()

# 전처리 전 데이터프레임
selected_reviews

,Review,Beer_name,MultinomialNB_label
0,Surprisingly little taste. It is fresh and a t...,Asahi Super Dry,Positive
1,Pours an almost honey color with an extremely ...,Asahi Super Dry,Positive
2,"Smell and taste of maltiness, grass and bread....",Asahi Super Dry,Positive
3,No redeeming features of this 'beer' other tha...,Asahi Super Dry,Negative
4,"At least, there's no harsh/off-flavours. What'...",Asahi Super Dry,Negative
...,...,...,...
2100,shrewd metal piece of work and a aroma. with d...,Asahi Super Dry,Negative
2101,freak color. it. equal compose Canadian-Molson...,Asahi Super Dry,Negative
2102,middling japanese beer weak/watery. suck.........,Asahi Super Dry,Negative
2103,"character. only thirsty. designate smell, big ...",Asahi Super Dry,Negative


time: 234 ms (started: 2023-08-28 21:57:22 +09:00)


In [7]:
# 전처리 후 데이터프레임
pre_apply_selected_reviews['Review'] = pre_apply_selected_reviews['Review'].apply(lambda x: preprocess_sentence(x))
pre_apply_selected_reviews

C:\Users\Gyeom\AppData\Local\Temp\ipykernel_14700\1098187607.py:44: FutureWarning: Possible nested set at position 23
  sentence = re.sub(r"bottle[ .,?!:]|bottled[[ .,?!:]]", "", sentence) # bottle 관련 삭제


,Review,Beer_name,MultinomialNB_label
0,surprisingly little taste. it is fresh and a t...,Asahi Super Dry,Positive
1,pours an almost honey color with an extremely ...,Asahi Super Dry,Positive
2,"smell and taste of maltiness, grass and bread....",Asahi Super Dry,Positive
3,no redeeming features of this 'beer' other tha...,Asahi Super Dry,Negative
4,"at least, there is no harsh off flavours. what...",Asahi Super Dry,Negative
...,...,...,...
2100,shrewd metal piece of work and a aroma. with d...,Asahi Super Dry,Negative
2101,freak color. it. equal compose canadian molson...,Asahi Super Dry,Negative
2102,middling japanese beer weak watery. suck,Asahi Super Dry,Negative
2103,"character. only thirsty. designate smell, big ...",Asahi Super Dry,Negative


time: 94 ms (started: 2023-08-28 21:57:29 +09:00)


---

## ■ KeyBERT 적용하여 키워드 추출

In [ ]:
# # 필요 라이브러리 설치
# !pip install keybert
# !pip install keyphrase-vectorizers
# !pip3 install -U spaCy
# !python -m spaCy download en_core_web_lg

In [8]:
# 필요 라이브러리 선언
import pandas as pd
import spacy

from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer # n_gram 대신 문맥, 문법을 고쳐주는 Vectorizer

time: 5.22 s (started: 2023-08-28 21:58:22 +09:00)


In [9]:
# KeyBERT 모델 생성 및 Pre-train model 객체를 내부에서 생성
kw_model = KeyBERT('all-mpnet-base-v2')
nlp = spacy.load('en_core_web_lg')

# 참고: https://github.com/TimSchopf/KeyphraseVectorizers

time: 2.06 s (started: 2023-08-28 22:00:59 +09:00)


<hr style="border: none; border-top: 2px dashed gray;"/>

- 기본 KeyphraseCountVectorizer 사용

In [10]:
def extract_keywords_for_beer(doc):
    try:
        keywords = kw_model.extract_keywords(doc, vectorizer=KeyphraseCountVectorizer(), top_n=5)
    except ValueError:
        # 키워드를 찾지 못한 경우 빈 리스트 반환
        keywords = []
    return [keyword for keyword, score in keywords]

print("[리뷰]")
print(pre_apply_beer_Wired_iStout['Review'].iloc[0])
k = extract_keywords_for_beer(pre_apply_beer_Wired_iStout['Review'].iloc[0])
print("\n[추출 키워드]")
print(*k, sep="\n")

[리뷰]
inky black beer , small head sweet molasses aroma , toasty malt , mocha coffee same on palate : choccy mocha taste , thick syrupy sipping goodness , lactose sweet milkiness : yummy

[추출 키워드]
mocha taste
mocha coffee
sweet molasses aroma
lactose sweet milkiness
inky black beer
time: 1.06 s (started: 2023-08-16 07:25:54 +09:00)


- 사용자 정의 vectorizer 사용

In [10]:
# Init vectorizer for the English language
# '<ADJ.*>*<N.*>+' extracts keywords that have 0 or more adjectives, followed by 1 or more nouns using the English spaCy part-of-speech tags.
vectorizer_1 = KeyphraseCountVectorizer(spacy_pipeline=nlp, pos_pattern='<ADJ>*<N.*>+', stop_words='english') # 형용사 + 최소 하나 이상의 명사
vectorizer_2 = KeyphraseCountVectorizer(spacy_pipeline=nlp, pos_pattern='(<N.*>+<N.*>+)', stop_words='english') # 명사 + 명사
vectorizer_3 = KeyphraseCountVectorizer(spacy_pipeline=nlp, pos_pattern='<JJ.*>+<NN.*>', stop_words='english') # 형용사 + 명사

time: 0 ns (started: 2023-08-28 22:01:18 +09:00)


In [12]:
# 사용자 정의 vectorizer 사용
def extract_keywords_for_beer_2(doc):
    try:
        keywords = kw_model.extract_keywords(doc, vectorizer=vectorizer_3, top_n=5)
    except ValueError:
        # 키워드를 찾지 못한 경우 빈 리스트 반환
        keywords = []
    return [keyword for keyword, score in keywords]


print("[리뷰]")
print(pre_apply_selected_reviews['Review'].iloc[0])
k = extract_keywords_for_beer_2(pre_apply_selected_reviews['Review'].iloc[0])
print("\n[추출 키워드: 사용자 정의 vectorizer]")
print(*k, sep="\n")

[리뷰]
surprisingly little taste. it is fresh and a typical lager, but more for thirst than taste

[추출 키워드: 사용자 정의 vectorizer]
little taste
typical lager
time: 94 ms (started: 2023-08-28 22:01:56 +09:00)


- 데이터 프레임에 KeyphraseCountVectorizer 적용

In [13]:
# 키워드 추출 함수 적용
pre_apply_selected_reviews['Keywords'] = pre_apply_selected_reviews['Review'].apply(extract_keywords_for_beer_2)

# 결과 출력
pre_apply_selected_reviews

,Review,Beer_name,MultinomialNB_label,Keywords
0,surprisingly little taste. it is fresh and a t...,Asahi Super Dry,Positive,"[little taste, typical lager]"
1,pours an almost honey color with an extremely ...,Asahi Super Dry,Positive,"[typical cheap beer, effervescent head, above ..."
2,"smell and taste of maltiness, grass and bread....",Asahi Super Dry,Positive,"[light bitterness, okay beer, entire mouth, mo..."
3,no redeeming features of this 'beer' other tha...,Asahi Super Dry,Negative,"[true beer, bland pale mass, commercial produc..."
4,"at least, there is no harsh off flavours. what...",Asahi Super Dry,Negative,"[harsh off flavours, sweet lager, dry finish]"
...,...,...,...,...
2100,shrewd metal piece of work and a aroma. with d...,Asahi Super Dry,Negative,"[oily hop, shrewd metal, watery finish, dry di..."
2101,freak color. it. equal compose canadian molson...,Asahi Super Dry,Negative,[golden worth]
2102,middling japanese beer weak watery. suck,Asahi Super Dry,Negative,"[middling japanese beer, weak watery]"
2103,"character. only thirsty. designate smell, big ...",Asahi Super Dry,Negative,"[designate smell, useless gold, adjunct ounce,..."


time: 2min 57s (started: 2023-08-28 22:02:09 +09:00)


In [14]:
pre_apply_selected_reviews.to_csv('pre_apply_selected_reviews_JJ_NN.csv', encoding='utf-8', index = False)

time: 16 ms (started: 2023-08-28 22:21:21 +09:00)


---

## ■ 키워드 집계 (동의어, 유의어 그룹핑)

- 키워드 추출된 데이터프레임 불러오기

In [2]:
cd "./Beer_Sentiment_analysis/Data"

c:\Users\Gyeom\OneDrive - 한국항공대학교\바탕 화면\머신러닝\프로젝트\맥주 측면 감정 분석\Beer_Sentiment_analysis\Data


In [3]:
import pandas as pd
pre_apply_beer_Wired_iStout = pd.read_csv('pre_apply_beer_Wired_iStout3_JJ_NN.csv')
pre_apply_beer_Wired_iStout

,Review,Beer_name,MultinomialNB_label,Keywords
0,"inky black beer , small head sweet molasses ar...",8 Wired iStout,Positive,"['lactose sweet milkiness', 'inky black beer',..."
1,draft. black beer with a tan head. chocolate a...,8 Wired iStout,Positive,"['black beer', 'light molasses', 'light nuttin..."
2,poured black color with a tan head. aroma of r...,8 Wired iStout,Positive,"['dark chocolate', 'roasted malt', 'subtle mal..."
3,"roasted dark chocolate aroma, beautiful black ...",8 Wired iStout,Positive,"['roasted dark chocolate', 'black licorice', '..."
4,old rating . pours black with a beige head. ar...,8 Wired iStout,Positive,"['moderate bitter finish', 'old rating', 'beig..."
...,...,...,...,...
397,from vinens verden. nice tan head with good du...,8 Wired iStout,Positive,"['nice tan', 'good duration']"
398,pours totally black with a very brown head. ar...,8 Wired iStout,Positive,"['brown head', 'dry smokey']"
399,tried again this beer sets the standard for au...,8 Wired iStout,Positive,"['australasian imperial stouts', 'best beer', ..."
400,my non rated did not use of goods and services...,8 Wired iStout,Negative,['present desire']


In [3]:
# 지정된 맥주 리뷰의 감정별 분리
df_po= pre_apply_beer_Wired_iStout[pre_apply_beer_Wired_iStout['MultinomialNB_label'] == "Positive"]
df_po.reset_index(drop = True,inplace = True)
df_po

,Review,Beer_name,MultinomialNB_label,Keywords
0,"inky black beer , small head sweet molasses ar...",8 Wired iStout,Positive,"['lactose sweet milkiness', 'inky black beer',..."
1,draft. black beer with a tan head. chocolate a...,8 Wired iStout,Positive,"['black beer', 'light molasses', 'light nuttin..."
2,poured black color with a tan head. aroma of r...,8 Wired iStout,Positive,"['dark chocolate', 'roasted malt', 'subtle mal..."
3,"roasted dark chocolate aroma, beautiful black ...",8 Wired iStout,Positive,"['roasted dark chocolate', 'black licorice', '..."
4,old rating . pours black with a beige head. ar...,8 Wired iStout,Positive,"['moderate bitter finish', 'old rating', 'beig..."
...,...,...,...,...
391,it poured an almost black color with a beige h...,8 Wired iStout,Positive,"['roasted malts', 'black color', 'beige head',..."
392,"can at home. very dark brown, fine foamy beige...",8 Wired iStout,Positive,"['dark chocolate', 'roasted malty', 'fine foam..."
393,from vinens verden. nice tan head with good du...,8 Wired iStout,Positive,"['nice tan', 'good duration']"
394,pours totally black with a very brown head. ar...,8 Wired iStout,Positive,"['brown head', 'dry smokey']"


In [4]:
df_ne= pre_apply_beer_Wired_iStout[pre_apply_beer_Wired_iStout['MultinomialNB_label'] == "Negative"]
df_ne.reset_index(drop = True,inplace = True)
df_ne

,Review,Beer_name,MultinomialNB_label,Keywords
0,no desire to write a review. this app now requ...,8 Wired iStout,Negative,[]
1,most import stouts i have a lot of trepidation...,8 Wired iStout,Negative,['most import']
2,"wow. very creamy, coffee, alc but you wouldn t...",8 Wired iStout,Negative,['good job']
3,i great tasting beer. nice and dark. a little ...,8 Wired iStout,Negative,[]
4,my non rated did not use of goods and services...,8 Wired iStout,Negative,['present desire']
5,and for short my axerophthol tasting nice beer...,8 Wired iStout,Negative,"['nice beer', 'great hoppy dark']"


<hr style="border: none; border-top: 2px dashed gray;"/>

- 유사도 그룹핑 방식의 집계 구현

In [5]:
from collections import defaultdict
import spacy

nlp = spacy.load('en_core_web_lg')

def merge_similar_keywords(dataframe, similarity_threshold=0.7):
    # # (csv파일을 읽은 데이터프레임일 때) 문자열로 표현된 리스트를 실제 리스트로 변환
    # dataframe['Keywords_List'] = dataframe['Keywords'].apply(lambda x: ast.literal_eval(x))
    
    # 모든 키워드 리스트를 하나로 합침
    all_keywords = dataframe['Keywords'].sum()

    initial_counts = defaultdict(int)
    for keyword in all_keywords:
        initial_counts[keyword] += 1

    merged_keywords = defaultdict(int)
    for keyword, count in initial_counts.items():
        doc1 = nlp(keyword)

        merged_or_found_similar = False
        for merged_keyword in list(merged_keywords):
            # 키워드가 일치하는 경우 바로 빈도 수를 올림
            if keyword == merged_keyword:
                merged_keywords[merged_keyword] += count
                merged_or_found_similar = True
                break

            doc2 = nlp(merged_keyword)
            if doc1.similarity(doc2) > similarity_threshold:
                merged_keywords[merged_keyword] += count
                merged_or_found_similar = True
                break

        if not merged_or_found_similar:
            merged_keywords[keyword] = count

    # # (csv파일을 읽은 데이터프레임일 때) 변환된 'Keywords_List' 컬럼 삭제
    # dataframe.drop('Keywords_List', axis=1, inplace=True)

    # 빈도 수 차등에 따라 정렬한 결과를 반환(내림차순)
    sorted_merged_keywords = sorted(merged_keywords.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_merged_keywords)


In [43]:
# 모든 리뷰의 'keywords' 컬럼에 있는 단어들을 카운트
filtered_and_merged_keywords = merge_similar_keywords(pre_apply_beer_Wired_iStout)
print(*filtered_and_merged_keywords.items(), sep="\n")

('dark chocolate', 178)
('tan head', 148)
('toasty malt', 127)
('bitter roast', 55)
('black color', 53)
('light molasses', 51)
('lactose sweet milkiness', 44)
('inky black beer', 38)
('nice sipper', 38)
('medium body', 35)
('moderate finish', 34)
('solid beer', 32)
('big stouts', 30)
('herbal aroma', 29)
('good roast', 27)
('thick syrupy', 22)
('interesting imperial stout', 20)
('medium density', 18)
('slight tartness', 15)
('smooth coffee', 13)
('black pour', 13)
('low carbonation', 13)
('right balance', 12)
('rich liquorice', 12)
('flat carbonation', 11)
('dry finish', 11)
('excellent head', 11)
('very coffee', 10)
('subtle vanilla', 9)
('milky notes', 9)
('heavy liquorice', 9)
('full caramel', 9)
('average carbonation', 8)
('oily textute', 7)
('huge coffee', 7)
('long aftertaste', 7)
('weak aroma', 7)
('fantastic chocolate', 7)
('milky mouthfeel', 6)
('best impy', 6)
('excellent stout', 6)
('roasty hop', 6)
('bitter side', 5)
('roasted seeds', 5)
('balanced brew', 5)
('small beige',

<hr style="border: none; border-top: 2px dashed gray;"/>

- 유사도 그룹핑 방식의 집계 구현 + __병합된 키워드 Mapping__

In [3]:
from collections import defaultdict
import spacy
import ast

nlp = spacy.load('en_core_web_lg')

def merge_similar_keywords(dataframe, similarity_threshold=0.9):    
    # (csv파일을 읽은 데이터프레임일 때) 문자열로 표현된 리스트를 실제 리스트로 변환
    # 데이터프레임 바로 쓸땐 아래 방법 쓰지 않고 'Keywords_List'가 아닌 'Keywords' 바로 쓰면 됨
    dataframe['Keywords_List'] = dataframe['Keywords'].apply(lambda x: ast.literal_eval(x))

    # 모든 키워드 리스트를 하나로 합침
    all_keywords = dataframe['Keywords_List'].sum()

    initial_counts = defaultdict(int)
    for keyword in all_keywords:
        initial_counts[keyword] += 1

    merged_keywords = defaultdict(int)
    keyword_mappings = defaultdict(list)
    for keyword, count in initial_counts.items():
        doc1 = nlp(keyword)

        merged_or_found_similar = False
        for merged_keyword in list(merged_keywords):
            if keyword == merged_keyword:
                merged_keywords[merged_keyword] += count
                merged_or_found_similar = True
                keyword_mappings[merged_keyword].append(keyword)
                break

            doc2 = nlp(merged_keyword)
            if doc1.similarity(doc2) > similarity_threshold:
                merged_keywords[merged_keyword] += count
                merged_or_found_similar = True
                keyword_mappings[merged_keyword].append(keyword)
                break

        if not merged_or_found_similar:
            merged_keywords[keyword] = count
            keyword_mappings[keyword] = [keyword]

    # (csv파일을 읽은 데이터프레임일 때) 변환된 'Keywords_List' 컬럼 삭제
    dataframe.drop('Keywords_List', axis=1, inplace=True)

    sorted_merged_keywords = sorted(merged_keywords.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_merged_keywords), dict(keyword_mappings)

filtered_and_merged_keywords, merged_keyword_mappings = merge_similar_keywords(pre_apply_beer_Wired_iStout)


In [13]:
# 어떤 키워드가 병합됐는지 매핑하여 시각적 확인
# print("▶ 병합된 키워드 집계:", filtered_and_merged_keywords)
print("▶ 병합된 키워드 매핑:")
# for i in [key for key, value in filtered_and_merged_keywords.items()]:
for key, value in filtered_and_merged_keywords.items():
    print(f"병합된 키워드: {key} [{value}]")
    print(merged_keyword_mappings[key])
    print()
    
# print("병합된 키워드 매핑:", merged_keyword_mappings)

▶ 병합된 키워드 매핑:
병합된 키워드: roasted malt [92]
['roasted malt', 'roasted malts', 'roasted malty', 'roasted sweet malt']

병합된 키워드: dark chocolate [73]
['dark chocolate', 'roasted dark chocolate', 'thick dark chocolate', 'bitter dark chocolate', 'sweet dark choc', 'super dark chocolate', 'dark bitter chocolate', 'tangy dark chocolate', 'dark cacao', 'dark berries', 'dark cherry', 'dark flavour', 'dark candy', 'dark fruity', 'dark cocoa', 'dark bittersweet', 'dark caramel', 'sweet dark chocolate', 'dark chocolat', 'intense dark chocolate']

병합된 키워드: tan head [35]
['tan head', 'thick tan head', 'medium tan head', 'decent tan head', 'creamy tan head', 'slight tan head', 'moderate tan head', 'little tan head', 'tan brown head']

병합된 키워드: black color [35]
['black color', 'black colour', 'dark black color', 'dark black colour', 'dark brown black color', 'strong dark black color']

병합된 키워드: brown head [34]
['brown head', 'sparse brown head', 'foamy brown head', 'dense brown head', 'creamy brown head'

<hr style="border: none; border-top: 2px dashed gray;"/>

### ★ __[최적화]__ 유사도 그룹핑 방식의 집계 구현 + 병합된 키워드 Mapping

In [6]:
from collections import defaultdict
import spacy
import ast

nlp = spacy.load('en_core_web_lg')

# 키워드의 초기 빈도 수를 계산
def get_initial_counts(keywords):
    initial_counts = defaultdict(int)
    for keyword in keywords:
        initial_counts[keyword] += 1
    return initial_counts

 # 두 개의 Doc 객체의 유사도를 계산하되 빈 벡터에 대한 처리를 함
def get_similarity(doc1, doc2):
    if doc1.has_vector and doc2.has_vector:
        return doc1.similarity(doc2)
    else:
        # 유사도 0으로 반환
        return 0

# 유사한 키워드들을 병합하고 키워드 빈도 수와 매핑 저장
def merge_keywords(keyword_counts, similarity_threshold):
    merged_keywords = defaultdict(int)
    keyword_mappings = defaultdict(list)
    keyword_docs = {}

    # 이미 등록된 키워드는 nlp 계산(임베딩)하지 않고 바로 리턴 (Memoization)
    def get_keyword_doc(keyword):
        if keyword not in keyword_docs:
            keyword_docs[keyword] = nlp(keyword)
        return keyword_docs[keyword]

    
    # 초기 빈도 딕셔너리를 순회하며 각 키워드에 대해 임베딩
    for keyword, count in keyword_counts.items():
        doc1 = get_keyword_doc(keyword)
        merged_or_found_similar = False

        # 이미 병합된 키워드 목록에서 해당 키워드와 비교
        for merged_keyword in list(merged_keywords):
            # 이미 병합된 키워드에 현재 키워드가 포함되어 있다면 빈도를 더함
            if keyword == merged_keyword:
                merged_keywords[merged_keyword] += count
                merged_or_found_similar = True
                keyword_mappings[merged_keyword].append(keyword)
                break

            # 기존 병합된 키워드와 현재 키워드의 임베딩을 비교하여 유사도를 계산
            doc2 = get_keyword_doc(merged_keyword)
            
            # 유사도가 임계값보다 큰 경우, 두 키워드를 같은 그룹으로 묶음
            if  get_similarity(doc1, doc2) > similarity_threshold:
                merged_keywords[merged_keyword] += count
                merged_or_found_similar = True
                keyword_mappings[merged_keyword].append(keyword)
                break

        # 현재 키워드가 기존 병합된 키워드 목록에 포함되지 않고, 유사한 키워드도 없다면 새로운 항목으로 추가
        if not merged_or_found_similar:
            merged_keywords[keyword] = count
            keyword_mappings[keyword] = [keyword]

    return merged_keywords, keyword_mappings

def merge_similar_keywords(dataframe, similarity_threshold=0.8):
    # CSV파일에서 문자열로 표현된 리스트를 실제 리스트로 변환
    dataframe['Keywords_List'] = dataframe['Keywords'].apply(lambda x: ast.literal_eval(x))
    all_keywords = dataframe['Keywords_List'].sum()

    # 빈도 수 계산 및 병합
    initial_counts = get_initial_counts(all_keywords)
    filtered_and_merged_keywords, merged_keyword_mappings = merge_keywords(initial_counts, similarity_threshold)
    sorted_merged_keywords = sorted(filtered_and_merged_keywords.items(), key=lambda x: x[1], reverse=True)

    # 변환된 'Keywords_List' 컬럼 삭제
    dataframe.drop('Keywords_List', axis=1, inplace=True)

    return dict(sorted_merged_keywords), dict(merged_keyword_mappings)

filtered_and_merged_keywords, merged_keyword_mappings = merge_similar_keywords(pre_apply_beer_Wired_iStout)


time: 17.7 s (started: 2023-08-19 09:49:04 +09:00)


In [7]:
# 어떤 키워드가 병합됐는지 매핑하여 시각적 확인
# print("▶ 병합된 키워드 집계:", filtered_and_merged_keywords)
print("▶ 병합된 키워드 그룹 개수:", len(filtered_and_merged_keywords))
print("▶ 병합된 키워드 매핑:")
for key, value in filtered_and_merged_keywords.items():
    print(f"병합된 키워드: {key} [{value}]")
    print(merged_keyword_mappings[key])
    print()

▶ 병합된 키워드 그룹 개수: 303
▶ 병합된 키워드 매핑:
병합된 키워드: dark chocolate [154]
['dark chocolate', 'roasted dark chocolate', 'bitter dark chocolate', 'sweet dark choc', 'dark brown colour', 'dark malt', 'dark brown bread', 'super dark chocolate', 'dark fruits', 'dark coffee', 'dark brow', 'dark bitter chocolate', 'dark roast', 'dark fruit', 'dark brown', 'tangy dark chocolate', 'dark dry chocolate', 'deep dark brown colour', 'bitter dark fruits', 'dark sugars', 'clear dark brown', 'dark bread', 'dark oily brown colour', 'sweet dark fruits', 'creamy dark brown head', 'dark malts', 'dark cacao', 'dark berries', 'dark colour', 'dark cherry', 'dark brown color', 'dark flavour', 'dark candy', 'dark fruity', 'clear dark brown colour', 'lighter dark fruits', 'dark ruby', 'dark roasted malt', 'dark cocoa', 'nice dark black', 'dark black colour', 'dark brown black color', 'dark bittersweet', 'thin dark beige lacing', 'dark roasted bitter malt', 'dark caramel', 'sweet dark chocolate', 'dark chocolat', 'dark co

<hr style="border: none; border-top: 2px dashed gray;"/>

# ★ 명사 위주의 유사도 그룹핑 실험

In [8]:
from collections import defaultdict
import spacy
import ast

nlp = spacy.load('en_core_web_lg')

# 키워드의 초기 빈도 수를 계산
def get_initial_counts(keywords):
    initial_counts = defaultdict(int)
    for keyword in keywords:
        initial_counts[keyword] += 1
    return initial_counts

# 명사 토큰 저장하고 각 유사성을 계산하여 명사 유사성 임계값을 초과하는지 확인
# 형용사의 차이는 개인차에 의해 발생하고 명사는 특징 자체를 나타내기에 집중 그룹핑 대상이 됨
def custom_similarity(doc1, doc2, noun_similarity_threshold):
    doc1_nouns = [token for token in doc1 if token.pos_ == 'NOUN']
    doc2_nouns = [token for token in doc2 if token.pos_ == 'NOUN']
    noun_similar = any(t1.similarity(t2) > noun_similarity_threshold for t1 in doc1_nouns for t2 in doc2_nouns)
    return noun_similar

# 유사한 키워드들을 병합하고 키워드 빈도 수와 매핑 저장
def merge_keywords(keyword_counts, noun_similarity_threshold):
    merged_keywords = defaultdict(int)
    keyword_mappings = defaultdict(list)
    keyword_docs = {}

    # 이미 등록된 키워드는 nlp 계산(임베딩)하지 않고 바로 리턴 (Memoization)
    def get_keyword_doc(keyword):
        if keyword not in keyword_docs:
            keyword_docs[keyword] = nlp(keyword)
        return keyword_docs[keyword]

    
    # 초기 빈도 딕셔너리를 순회하며 각 키워드에 대해 임베딩
    for keyword, count in keyword_counts.items():
        doc1 = get_keyword_doc(keyword)
        merged_or_found_similar = False

        # 이미 병합된 키워드 목록에서 해당 키워드와 비교
        for merged_keyword in list(merged_keywords):
            # 이미 병합된 키워드에 현재 키워드가 포함되어 있다면 빈도를 더함
            if keyword == merged_keyword:
                merged_keywords[merged_keyword] += count
                merged_or_found_similar = True
                keyword_mappings[merged_keyword].append(keyword)
                break

            doc2 = get_keyword_doc(merged_keyword)
            
            # 유사도가 임계값보다 큰 경우, 두 키워드를 같은 그룹으로 묶음
            if custom_similarity(doc1, doc2, noun_similarity_threshold):
                merged_keywords[merged_keyword] += count
                merged_or_found_similar = True
                keyword_mappings[merged_keyword].append(keyword)
                break

        # 현재 키워드가 기존 병합된 키워드 목록에 포함되지 않고, 유사한 키워드도 없다면 새로운 항목으로 추가
        if not merged_or_found_similar:
            merged_keywords[keyword] = count
            keyword_mappings[keyword] = [keyword]

    return merged_keywords, keyword_mappings

def merge_similar_keywords_noun_weight(dataframe, noun_similarity_threshold=0.7):
    # 문자열로 표현된 리스트를 실제 리스트로 변환
    dataframe['Keywords_List'] = dataframe['Keywords'].apply(lambda x: ast.literal_eval(x))
    all_keywords = dataframe['Keywords_List'].sum()

    # 빈도 수 계산 및 병합
    initial_counts = get_initial_counts(all_keywords)
    filtered_and_merged_keywords, merged_keyword_mappings = merge_keywords(initial_counts, noun_similarity_threshold)
    sorted_merged_keywords = sorted(filtered_and_merged_keywords.items(), key=lambda x: x[1], reverse=True)

    # 변환된 'Keywords_List' 컬럼 삭제
    dataframe.drop('Keywords_List', axis=1, inplace=True)

    return dict(sorted_merged_keywords), dict(merged_keyword_mappings)


filtered_and_merged_keywords, merged_keyword_mappings = merge_similar_keywords_noun_weight(pre_apply_beer_Wired_iStout)

C:\Users\Gyeom\AppData\Local\Temp\ipykernel_28176\1564491153.py:17: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  noun_similar = any(t1.similarity(t2) > noun_similarity_threshold for t1 in doc1_nouns for t2 in doc2_nouns)


In [9]:
# 어떤 키워드가 병합됐는지 매핑하여 시각적 확인
# print("▶ 병합된 키워드 집계:", filtered_and_merged_keywords)
print("▶ 병합된 키워드 그룹 개수:", len(filtered_and_merged_keywords))
print("▶ 병합된 키워드 매핑:")
for key, value in filtered_and_merged_keywords.items():
    print(f"병합된 키워드: {key} [{value}]")
    print(merged_keyword_mappings[key])
    print()

▶ 병합된 키워드 그룹 개수: 222
▶ 병합된 키워드 매핑:
병합된 키워드: tan head [185]
['tan head', 'beige head', 'large foamy beige head', 'good brown head', 'brown head', 'thick tan head', 'small brown head', 'sparse brown head', 'less head', 'medium head', 'small beige head', 'light brown head', 'decent head', 'excellent head', 'medium sized head', 'medium beige head', 'creamy head', 'thin tan head', 'small brownish head', 'dark beige head', 'brownish head', 'small head', 'nice beige brown head', 'tiny brief head', 'small tan head', 'fine thin head', 'medium tan head', 'tiny beige head', 'decent tan head', 'khaki head', 'creamy dark brown head', 'foamy brown head', 'medium sized brown head', 'low brown head', 'little head', 'creamy tan head', 'dense brown head', 'light tan head', 'much head', 'white head', 'creamy brown head', 'nice light head', 'tiny brown head', 'slight tan head', 'dark tanm head', 'medium brown head', 'small dark brown head', 'small unstable beige head', 'off white head', 'dark brown head',